In [1]:
import numpy as np
from einops import rearrange

In [24]:
from PIL import Image
from pathlib import Path
from tqdm import tqdm

In [48]:
!ls shader-park-core.appspot.com/

sculptureThumbnails


In [14]:
path = Path("shader-park-core.appspot.com/sculptureThumbnails")

In [49]:
dim = 34
img_size = 512
total_dim = dim*img_size
main_img = np.zeros((total_dim,total_dim,4), dtype=np.uint8)
x_idx = 0
y_idx = 0
all_paths = list(path.glob("*.jpeg")) + list(path.glob("*.png"))
for p in tqdm(all_paths):
    img = Image.open(p)
    resized_img = img.resize((img_size, img_size))
    if (resized_img.mode != "RGBA"):
        new_img = np.ones((img_size,img_size,4), dtype=np.uint8)*255
        new_img[:,:,:3] = resized_img
        # from https://github.com/PWhiddy/PokemonRedExperiments/blob/master/MapWalkingVis.ipynb
        alpha_val = np.array([255, 255,  255, 255], dtype=np.uint8)
        alpha_mask = (new_img == alpha_val).all(axis=2).reshape(img_size,img_size,1)
        resized_img = np.where(alpha_mask, np.array([[[0,0,0,0]]]), new_img)
    main_img[
        x_idx*img_size:(x_idx+1)*img_size, 
        y_idx*img_size:(y_idx+1)*img_size] = np.asarray(resized_img)
    x_idx += 1
    if x_idx >= dim:
        x_idx = 0
        y_idx += 1
im = Image.fromarray(main_img)
im.save("grid_full.png")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1125/1125 [00:39<00:00, 28.21it/s]


In [50]:
total_dim

17408

In [63]:
# cropped version
dim = 34
pre_crop = 512
img_size = 256
crop_amt = (pre_crop - img_size) / 2
total_dim = dim*img_size
main_img = np.zeros((total_dim,total_dim,4), dtype=np.uint8)
x_idx = 0
y_idx = 0
all_paths = list(path.glob("*.jpeg")) + list(path.glob("*.png"))
#all_paths = all_paths[:20]
for p in tqdm(all_paths):
    img = Image.open(p)
    resized_img = img.resize((pre_crop, pre_crop))
    if (resized_img.mode != "RGBA"):
        new_img = np.ones((pre_crop,pre_crop,4), dtype=np.uint8)*255
        new_img[:,:,:3] = resized_img
        # from https://github.com/PWhiddy/PokemonRedExperiments/blob/master/MapWalkingVis.ipynb
        alpha_val = np.array([255, 255,  255, 255], dtype=np.uint8)
        alpha_mask = (new_img == alpha_val).all(axis=2).reshape(pre_crop,pre_crop,1)
        resized_img = Image.fromarray( np.where(alpha_mask, np.array([[[0,0,0,0]]]), new_img).astype(np.uint8) )
    #print((crop_amt,pre_crop-crop_amt,pre_crop-crop_amt, crop_amt))
    # left, top, right, bottom
    cropped_img = resized_img.crop((crop_amt,crop_amt,pre_crop-crop_amt, pre_crop-crop_amt))
    main_img[
        x_idx*img_size:(x_idx+1)*img_size, 
        y_idx*img_size:(y_idx+1)*img_size] = np.asarray(cropped_img)
    x_idx += 1
    if x_idx >= dim:
        x_idx = 0
        y_idx += 1
im = Image.fromarray(main_img)
im.save("grid_cropped.png")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1125/1125 [00:38<00:00, 28.90it/s]
